In [331]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [335]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [337]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [338]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'): 
        anuncio_list.append(info.get_text())
    #Delete \n
    #anuncio_list[5] = anuncio_list[5][1:-1]
    #anuncio_list[6] = anuncio_list[6][1:-2]
    #anuncio_list[7] = anuncio_list[7][1:-1]
    return anuncio_list

In [339]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [340]:
def get_name_empresa(result_page):
    #response = get_page(trabajo_link)
    #result_page = BeautifulSoup(response.content,'lxml')
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    #for paragraph in hola.find('strong'):
    #    print(paragraph)
    return Enterprise_information.find('strong').get_text()

In [341]:
def clean_text(text):
    import unidecode
    text = text.lower()
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [342]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [184]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/688703/Analista-de-Compras---Santa-Cruz.html?searchId=1545853046.74&page=1"

In [333]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
#description_list = get_job_description(result_page)
#empresa_name = get_name_empresa(result_page)
#output = unify_lists([empresa_name, anuncio_list, description_list])

NameError: name 'get_page' is not defined

In [343]:
get_job_info(trabajo_link)

['NOSSAMODA SRL',
 'ID Empleo: 695472',
 'Ubicación: Santa Cruz de la Sierra, Bolivia',
 'Categoría: Contabilidad y Economía ',
 'Sueldo: De Bs. 3.000 a Bs. 3.999 ',
 'Contrato: Tiempo Completo',
 '\nPublicado: 19 Diciembre 2018\n',
 '\nVencimiento: 18 Enero 2019\n\n',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office ex

In [269]:
text

'analista de contabilidad formacion academica titulados en contabilidad o auditoria funciones  recepcion y revision de facturas de compras preparacion de libros de compras y ventas iva bancarizacion y planilla tributaria registros contables de transacciones diarias documentos de soporte cierres mensuales atencion requerimientos internos y externos  preparacion y analisis de estados de cuentas contables  elaboracion de planillas de sueldos aportes a las afps y caja de salud  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word  requisitos  organizado con capacidad analitica  buenas relaciones interpersonales  trabajo en equipo  experiencia experiencia minima de 2 anos en cargos similares dentro del rubro comercial  las personas interesadas que cumplan con el perfil podran remitir su curriculum vitae indicando su pretension salarial por trabajopolisbo'

In [345]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job_description + "&page=1&listings_per_page=100"
    
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        #trabajo_list = list()
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_link)
         #   trabajo_list.append((trabajo_name,trabajo_link))
        #for trabajo in trabajo_list:
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
        #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [346]:
Results = trabajopolis("tesorería")

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/689938/Jefe-de-Contabilidad-y-Tesorería.html?searchId=1545872212.12&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545872212.12&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/694359/Analista-de-Tesorería---La-Paz.html?searchId=1545872212.12&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/690744/Asistente-de-Tesorería.html?searchId=1545872212.12&page=1


In [348]:
import csv
outfile = open("/Users/Roberto/Documents/Online Learning/Python programming/Trabajopolis scraper/trabajo.csv", 'w',newline='')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        for cell in Results[i][0]:
            #print(cell)
            csvRow.append(cell)
        csvRow.append(Results[i][1])
        csvRow.append(Results[i][2])
        print(csvRow)
        writer.writerow(csvRow)
finally:
    outfile.close() 

['Importante Empresa Industrial', 'ID Empleo: 689938', 'Ubicación: Santa Cruz de la Sierra, Bolivia', 'Categoría: Contabilidad y Economía ', 'Sueldo: No Declarado ', 'Contrato: Tiempo Completo', '\nPublicado: 21 Diciembre 2018\n', '\nVencimiento: 20 Enero 2019\n\n', '\nPrimera Publicación: 28 Noviembre 2018\n', 'jefe de contabilidad y tesoreria\nobjetivo del cargogestionar los procesos contables de la empresa a fin de preparar informacion financiera y economica confiable y oportuna asi como dar solucion a problemas concernientes a la elaboracion control y administracion de dicha informacion de acuerdo con principios tecnicos normas legales y tributarias\nprincipales funciones preparar estados financieros analizar y recomendar mejoras a los diferentes procesos que se desarrollan en la empresa con base en las evaluaciones economicasfinancieras preparar y emitir comprobantes de egresos ingresos y traspasos de acuerdo con la realidad economica de las transacciones realizadas  autorizar pro